Run cell (commands) below and then hit F5 (refresh the page) right after that

In [ ]:
!npm install -g --unsafe-perm ijavascript
!ijsinstall --install=global
!jupyter-kernelspec list

# prepa

In [3]:
var { spawnSync } = require('child_process');
var sh = (cmd) => { 
    let res = spawnSync(cmd, { cwd: process.cwd(), stdio: 'pipe', shell: true, encoding: 'utf-8' })
    console.log(...res.output);
};
var run_async = async (pf) => {
  $$.async();
  await pf();
  $$.done();
};
sh('npm init -y');
sh('node -v; npm -v');

null Wrote to /content/package.json:

{
  "name": "content",
  "version": "1.0.0",
  "main": "index.js",
  "scripts": {
    "test": "echo \"Error: no test specified\" && exit 1"
  },
  "keywords": [],
  "author": "",
  "license": "ISC",
  "dependencies": {
    "@tensorflow/tfjs-node-gpu": "^3.6.1",
    "csv": "^5.5.0",
    "sharp": "^0.28.2"
  },
  "devDependencies": {},
  "description": ""
}


 
null v14.16.0
6.14.8
 


In [4]:
{
  let fs = require('fs')
  fs.mkdirSync('/root/.kaggle/', { recursive: true })
  fs.copyFileSync('kaggle.json', '/root/.kaggle/kaggle.json')
  sh('kaggle competitions download -c digit-recognizer')
  sh('unzip train.csv.zip')
  sh('unzip test.csv.zip')
  sh('npm install @tensorflow/tfjs-node-gpu')
  sh('npm install sharp')
  sh('npm install csv')
}

null Warning: Your Kaggle API key is readable by other users on this system! To fix this, you can run 'chmod 600 /root/.kaggle/kaggle.json'
test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
 
null Archive:  train.csv.zip
 replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)

null Archive:  test.csv.zip
 replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)

null 
> @tensorflow/tfjs-node-gpu@3.6.1 install /content/node_modules/@tensorflow/tfjs-node-gpu
> node scripts/install.js gpu download

GPU-linux-3.6.1.tar.gz
+ @tensorflow/tfjs-node-gpu@3.6.1
updated 1 package and audited 168 packages in 17.049s

6 packages are looking for funding

In [23]:
//var sharp = require('sharp')
var tf = require('@tensorflow/tfjs-node-gpu');
var csv = require('csv/lib/sync')
var sharp = require('sharp/lib/index')

$$.async();
process.on('unhandledRejection', error => {
  console.log(error);
  $$.done();
  process.exit(1)
 
});
(async () => {
  let data = csv.parse(fs.readFileSync('./train.csv')).slice(1);
  global._dataset = data.map(d => {
    let out = new Float32Array(10)
    out[parseInt(d[0])] = 1;

    let inp = new Float32Array(d.slice(1).map(v=>parseInt(v)))
    return [
      tf.tensor2d(out, [1, 10]),
      tf.tensor4d(inp,[1,28,28,1]).div(255)
    ]
  })
  let test = csv.parse(fs.readFileSync('./test.csv')).slice(1);
  global._datasetTest = test.map(d => {
    let inp = new Float32Array(d.map(v => parseInt(v)))
    return tf.tensor4d(inp, [1, 28, 28, 1]).div(255)
  })
  
  $$.png(
    Buffer.from(
      await sharp(
        Buffer.from(global._dataset[0][1].mul(255).dataSync()),
        { raw: { width: 28, height: 28, channels: 1 } }).png().toBuffer()
    ).toString("base64"))
  $$.done()
})()

# train

model architecture

In [140]:
var tf = require('@tensorflow/tfjs-node-gpu');
var model = tf.sequential();
model.add(tf.layers.dropout({rate:0.1, inputShape: [28,28, 1]}));
model.add(tf.layers.conv2d({ filters: 16, kernelSize: [7, 7], kernelInitializer: 'glorotUniform', activation: 'linear', padding: 'valid', useBias: true}));
model.add(tf.layers.leakyReLU());
model.add(tf.layers.conv2d({ filters: 20, kernelSize: [7, 7], kernelInitializer: 'glorotUniform', activation: 'linear', padding: 'valid', useBias: true }));
model.add(tf.layers.leakyReLU());
model.add(tf.layers.conv2d({ filters: 24, kernelSize: [7, 7], kernelInitializer: 'glorotUniform', activation: 'linear', padding: 'valid', useBias: true }));
model.add(tf.layers.leakyReLU());
model.add(tf.layers.conv2d({ filters: 3, kernelSize: [5, 5], kernelInitializer: 'glorotUniform', activation: 'linear', padding: 'valid', useBias: true }));
model.add(tf.layers.leakyReLU());
model.add(tf.layers.reshape({ targetShape: [6 * 6 * 3] }));
model.add(tf.layers.dense({ units: 10, kernelInitializer: 'glorotUniform', activation: 'linear', padding: 'valid', useBias: true }));
model.add(tf.layers.softmax());

model.summary()

_________________________________________________________________
Layer (type)                 Output shape              Param #   
dropout_Dropout3 (Dropout)   [null,28,28,1]            0         
_________________________________________________________________
conv2d_Conv2D9 (Conv2D)      [null,22,22,16]           800       
_________________________________________________________________
leaky_re_lu_LeakyReLU9 (Leak [null,22,22,16]           0         
_________________________________________________________________
conv2d_Conv2D10 (Conv2D)     [null,16,16,20]           15700     
_________________________________________________________________
leaky_re_lu_LeakyReLU10 (Lea [null,16,16,20]           0         
_________________________________________________________________
conv2d_Conv2D11 (Conv2D)     [null,10,10,24]           23544     
_________________________________________________________________
leaky_re_lu_LeakyReLU11 (Lea [null,10,10,24]           0         
__________

remove old models

In [142]:

fs.readdirSync('./models').forEach(fileName => {
  fs.rmdirSync('./models/'+fileName, { recursive: true })
})

train and save models to "models" folder

In [ ]:
var tf = require('@tensorflow/tfjs-node-gpu');

var validK = 0.99
var trainFiles = global._dataset.slice(0, Math.floor(global._dataset.length * validK))
var validFiles = global._dataset.slice(Math.ceil(global._dataset.length * validK))

global.getImages = (files, index, batchSize) => {
 
    let input = [];
  let output = [];
    for (let i = 0; i !== batchSize; i++) {
        input.push(files[index + i][1]);
        output.push(files[index + i][0]);
    }
    return { input: tf.concat(input), output: tf.concat(output)}
}
 
global.getValidation = async (model) => {
  let errorL = tf.scalar(0);
  let hit = 0;
    for (let i = 0; i !== validFiles.length; i++) {
        errorL = errorL.add(tf.tidy(() => {
        let { input, output } = global.getImages(validFiles, i, 1)
        
          let result = model.predict(input);
             let data = result.dataSync();
      let maxV = -Infinity;
      let maxI = 0;
      data.forEach((v,i) => {
        if (v > maxV) {
          maxV = v;
          maxI = i;
        }
      })
          if (output.dataSync()[maxI]||0===1) {
            hit++;
      }
            return loss(result, output)
        }))
    }
    return [errorL.div(validFiles.length).dataSync()[0],hit/validFiles.length];
}
 
global.minimize = (input, output) => {
        optimizer.minimize(() => {
          let result = model.predict(input);
            
            return loss(result, output)
        })
}
global.optimizer = tf.train.adam(3.0e-4);
global.loss = tf.losses.meanSquaredError;
 
global.main = async () => {
    let epochs = 50;
  let batchSize = 512;
    
    
    for (let e = 0; e !== epochs; e++) {
     
        let time = new Date().valueOf();
        for (let i = 0; i !== Math.floor(trainFiles.length / batchSize) * batchSize; i += batchSize) {
 
        tf.tidy(() => {
            //console.log(i);
            let { input, output } = global.getImages(trainFiles, i, batchSize);
            minimize(input, output);

        })
        }
         console.log(e, Date.now()- time)
        let shuffle = (array) => array.sort(() => Math.random() - 0.5);
        trainFiles = shuffle(trainFiles)
        let valError = await global.getValidation(model)
      console.log('valError: ', valError);
         try {
    await fs.promises.mkdir('models/');
  }catch(err){}
        model.save(`file://./models/${e}_${valError[0]}`)
    }


 
}
global.main();

0 9492
valError:  [ 0.0035864408127963543, 0.9761904761904762 ]
1 9174
valError:  [ 0.0017407124396413565, 0.9928571428571429 ]
2 9283
valError:  [ 0.003185026813298464, 0.9761904761904762 ]
3 9119
valError:  [ 0.0018053409876301885, 0.9880952380952381 ]
4 9299
valError:  [ 0.0037724534049630165, 0.9785714285714285 ]
5 9079
valError:  [ 0.002384751569479704, 0.9880952380952381 ]
6 9114
valError:  [ 0.002425662474706769, 0.9833333333333333 ]
7 9601
valError:  [ 0.002873930148780346, 0.9857142857142858 ]
8 9177
valError:  [ 0.0024221199564635754, 0.9833333333333333 ]
9 9068
valError:  [ 0.001903802971355617, 0.9880952380952381 ]
10 9205
valError:  [ 0.003074571955949068, 0.9761904761904762 ]


Test best model

In [157]:
(async () => {
  //get best model
  let testModel = await tf.loadLayersModel('file://./models/' + fs.readdirSync('./models').sort((l, r) => parseFloat(l.split('_')[1]) - parseFloat(r.split('_')[1]))[0] + '/model.json');
  let valError = await global.getValidation(testModel)
  console.log(valError)
})()

[ 0.0016014677239581943, 0.9952380952380953 ]


Generate csv file for competition

In [159]:
(async () => {
  //get best model
  let testModel = await tf.loadLayersModel('file://./models/' + fs.readdirSync('./models').sort((l, r) => parseFloat(l.split('_')[1]) - parseFloat(r.split('_')[1]))[0] + '/model.json');

  let out =[['ImageId','Label']]
  global._datasetTest.forEach((v, ImageId) => {
    tf.tidy(() => {

      let data = testModel.predict(v).dataSync();
      let maxV = -Infinity;
      let maxI = 0;
      data.forEach((v,i) => {
        if (v > maxV) {
          maxV = v;
          maxI = i;
        }
      })
      out.push([String(ImageId+1),String(maxI)])

     })
   })
  fs.writeFileSync('out.csv',csv.stringify(out))

})()

In [162]:
sh(`kaggle competitions submit -c digit-recognizer -f out.csv -m "tfjs"`)

null Warning: Your Kaggle API key is readable by other users on this system! To fix this, you can run 'chmod 600 /root/.kaggle/kaggle.json'
100%|##########| 208k/208k [00:08<00:00, 24.2kB/s]

